mediapipe 설치

In [ ]:
pip install pytube opencv-python mediapipe numpy


### 동영상에서 mediapipe 이용해서 keypoint 검출하기

📌 1. YouTube Shorts 다운로드

In [8]:
from pytube import YouTube

# 저장할 경로 설정
save_path = r"E:\Desktop\AI_learning\Project_2_Object_detection"
video_filename = "dog_video.mp4"
video_full_path = f"{save_path}\\{video_filename}"

# YouTube Shorts URL
video_url = "https://www.youtube.com/watch?v=ogzPdZxI4Jo"

try:
    # YouTube 영상 다운로드
    yt = YouTube(video_url)
    stream = yt.streams.get_highest_resolution()
    stream.download(output_path=save_path, filename=video_filename)
    print(f"동영상 다운로드 완료: {video_full_path}")

except Exception as e:
    print(f"다운로드 오류 발생: {e}")


다운로드 오류 발생: HTTP Error 403: Forbidden


✅ 2. 동영상에서 강아지 Keypoint 검출 코드 (MediaPipe)

In [1]:
import cv2
import mediapipe as mp

# 저장된 비디오 파일 경로
video_full_path = r"E:\Desktop\AI_learning\Project_2_Object_detection\dog_video.mp4"

# MediaPipe Pose 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, smooth_landmarks=True)

# 비디오 파일 불러오기
cap = cv2.VideoCapture(video_full_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # OpenCV BGR → RGB 변환 (MediaPipe는 RGB 입력 필요)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # MediaPipe 실행
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            h, w, _ = frame.shape
            x, y = int(landmark.x * w), int(landmark.y * h)
            cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)  # Keypoint 시각화

    cv2.imshow("Dog Pose Detection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


https://www.youtube.com/shorts/ogzPdZxI4Jo?feature=share

3. 이미지에서 - 강아지 Keypoint 검출 코드 mediapipe

In [2]:
import cv2
import mediapipe as mp
import os

# MediaPipe Pose 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=1, smooth_landmarks=True)

# 이미지 폴더 경로 설정
image_folder = r"E:\Desktop\AI_learning\Project_2_Object_detection\dog_images"
image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))])

# 이미지 11장 처리
for image_file in image_files[:11]:
    image_path = os.path.join(image_folder, image_file)
    
    # 이미지 읽기
    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지를 불러올 수 없음: {image_path}")
        continue

    # OpenCV BGR → RGB 변환
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # MediaPipe 실행
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        h, w, _ = image.shape
        for landmark in results.pose_landmarks.landmark:
            x, y = int(landmark.x * w), int(landmark.y * h)
            cv2.circle(image, (x, y), 5, (0, 255, 0), -1)  # Keypoint 시각화

    # 결과 이미지 출력
    cv2.imshow(f"Dog Keypoints - {image_file}", image)
    cv2.waitKey(0)  # 키 입력 대기 후 다음 이미지 표시

cv2.destroyAllWindows()


### 📌 Yolo - pose 모델로 - 강아지 객체 탐지 후 keypoint 찾는 코드


In [ ]:
pip install ultralytics

📌 YOLO-Pose 실행해보기

In [1]:
from ultralytics import YOLO
import cv2

# YOLO-Pose 모델 로드 (Pose 전용 모델 사용)
model = YOLO("yolov8x-pose.pt")  # YOLOv8-Pose 모델 사용

# 테스트할 이미지 경로
image_path = r"E:\Desktop\AI_learning\Project_2_Object_detection\dog_images\download.jpg"

# YOLO-Pose 실행 (Bounding Box + Keypoint 검출)
results = model(image_path)

# 결과 표시
for result in results:
    result.show()



image 1/1 E:\Desktop\AI_learning\Project_2_Object_detection\dog_images\download.jpg: 384x640 4 persons, 558.4ms
Speed: 1.0ms preprocess, 558.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)


📌 YOLO-Pose 적용하기 - 이미지 폴더 경로만 바꿔서 사용 , 11장 사용

In [ ]:
# 욜로 모델 종류

# YOLO11n-pose.pt: 가장 작은 모델로, 빠른 속도와 낮은 연산량을 특징으로 합니다.​
# YOLO11s-pose.pt: 소형 모델로, 속도와 정확도의 균형을 제공합니다.​
# YOLO11m-pose.pt: 중간 크기의 모델로, 향상된 정확도를 제공합니다.​
# YOLO11l-pose.pt: 대형 모델로, 높은 정확도를 제공합니다.​
# YOLO11x-pose.pt: 가장 큰 모델로, 최고의 정확도를 제공합니다.

# 자매품
# YOLO8n-pose.pt 시리즈 

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import os

# YOLO-Pose 모델 로드
model = YOLO("yolo11n-pose.pt")  # YOLOv8-Pose 모델 사용

# 이미지 폴더 경로
image_folder = r"E:\Desktop\AI_learning\Project_2_Object_detection\dog_images" # 
image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))])[:11]  # 11장 사용

# YOLO-Pose 실행 및 결과 시각화
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    image = cv2.imread(image_path)

    if image is None:
        print(f"이미지를 불러올 수 없음: {image_path}")
        continue

    # YOLO-Pose 실행
    results = model(image_path)

    for result in results:
        for box, keypoints in zip(result.boxes.xyxy, result.keypoints.xy):
            x1, y1, x2, y2 = map(int, box)  # Bounding Box 좌표

            # Bounding Box 그리기
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(image, "Dog", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            # Keypoint 그리기
            for x, y in keypoints:
                if x > 0 and y > 0:  # 유효한 좌표만 그림
                    cv2.circle(image, (int(x), int(y)), 5, (0, 255, 0), -1)

    # 결과 이미지 출력
    cv2.imshow(f"YOLO-Pose Keypoints - {image_file}", image)
    cv2.waitKey(0)  # 키 입력 후 다음 이미지 표시

cv2.destroyAllWindows()



image 1/1 E:\Desktop\AI_learning\Project_2_Object_detection\dog_images\download.jpg: 384x640 4 persons, 93.9ms
Speed: 2.5ms preprocess, 93.9ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 E:\Desktop\AI_learning\Project_2_Object_detection\dog_images\n02089973_1957.jpg: 640x608 (no detections), 79.0ms
Speed: 2.0ms preprocess, 79.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 608)

image 1/1 E:\Desktop\AI_learning\Project_2_Object_detection\dog_images\n02089973_2331.jpg: 576x640 (no detections), 74.1ms
Speed: 2.0ms preprocess, 74.1ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

image 1/1 E:\Desktop\AI_learning\Project_2_Object_detection\dog_images\n02089973_846.jpg: 576x640 1 person, 59.2ms
Speed: 2.1ms preprocess, 59.2ms inference, 1.0ms postprocess per image at shape (1, 3, 576, 640)

image 1/1 E:\Desktop\AI_learning\Project_2_Object_detection\dog_images\n02090379_2420.jpg: 480x640 (no detections), 63.7ms
Speed: 3.1ms